In [162]:
import pandas as pd

In [163]:
school_data_file = "Resources/schools_complete.csv"
student_data_file = "Resources/students_complete.csv"

In [164]:
school_data = pd.read_csv(school_data_file)
student_data = pd.read_csv(student_data_file)

In [165]:
merged = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
merged.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [166]:
schools = merged["school_name"].unique()
print(schools)

['Huang High School' 'Figueroa High School' 'Shelton High School'
 'Hernandez High School' 'Griffin High School' 'Wilson High School'
 'Cabrera High School' 'Bailey High School' 'Holden High School'
 'Pena High School' 'Wright High School' 'Rodriguez High School'
 'Johnson High School' 'Ford High School' 'Thomas High School']


In [167]:
num_schools = int(len(schools))
print(num_schools)

15


In [168]:
num_students = int(merged["student_name"].count())
print(num_students)

39170


In [169]:
budget = merged["budget"].unique()
total_budget = float(sum(budget))
print(total_budget)

24649428.0


In [170]:
math = merged["math_score"]
math_score = float(math.mean())
print(math_score)

78.98537145774827


In [171]:
reading = merged["reading_score"]
reading_score = float(reading.mean())
print(reading_score)

81.87784018381414


In [172]:
passing_math = merged.loc[(math >= 70), :]
pass_math = passing_math["Student ID"].count()

In [173]:
passing_reading = merged.loc[(reading >= 70), :]
pass_read = passing_reading["Student ID"].count()

In [174]:
percent_pass_math = (pass_math / num_students) * 100
print(percent_pass_math)

74.9808526933878


In [175]:
percent_pass_read = (pass_read / num_students) * 100
print(percent_pass_read)

85.80546336482001


In [176]:
overall_passing = merged.loc[(math >= 70) & (reading >= 70)]
overall_passing["Student ID"].head()
overall_passing_num = overall_passing["Student ID"].count()
percent_passing = (overall_passing_num / num_students) * 100
print(percent_passing)

65.17232575950983


In [187]:
district_summary = {
    "Total Schools": num_schools, 
    "Total Students": num_students, 
    "Total Budget": total_budget,
    "Average Math Score": math_score,
    "Average Reading Score": reading_score,
    "Percent Passing Math": percent_pass_math,
    "Percent Passing Reading": percent_pass_read,
    "Percent Passing Overall": percent_passing
}
district_summary_df = pd.DataFrame(district_summary, index=[0])
district_summary_df.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
0,15,39170,24649428.0,78.985371,81.87784,74.980853,85.805463,65.172326


In [188]:
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary_df.head()

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326
